#### TASK 2

In [1]:
pip install opencv-python imageio pillow keras tensorflow scikit-learn


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.8 MB 4.1 MB/s eta 0:00:10
    --------------------------------------- 0.6/38.8 MB 7.7 MB/s eta 0:00:05
   - -------------------------------------- 1.3/38.8 MB 10.3 MB/s eta 0:00:04
   -- ------------------------------------- 2.0/38.8 MB 11.8 MB/s eta 0:00:04
   -- ------------------------------------- 2.8/38.8 MB 13.9 MB/s eta 0:00:03
   --- ------------------------------------ 3.8/38.8 MB 14.2 MB/s eta 0:00:03
   ---- ----------------------------------- 4.3/38.8 MB 13.8 MB/s eta 0:00:03
   ----- ---------------------------------- 5.2/38.8 MB 14.4 MB/s eta 0:00:03
   ------ --------------------------------- 6.1/38.8 MB 15.0 MB/s eta 0:00:03
   ------ --------------------------------- 6.6/38.8 MB 14.5 MB/s eta 0:00:03


In [2]:
import os
import numpy as np
import imageio
from PIL import Image
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt


In [16]:
def extract_frames_from_tiff_folder(tiff_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all TIFF files in the folder
    for tiff_file in os.listdir(tiff_folder):
        if tiff_file.lower().endswith('.tiff') or tiff_file.lower().endswith('.tif'):
            tiff_path = os.path.join(tiff_folder, tiff_file)
            print(f'Processing file: {tiff_path}')
            
            # Extract frames from the current TIFF file
            with imageio.get_reader(tiff_path) as reader:
                for i, frame in enumerate(reader):
                    frame_filename = f'{os.path.splitext(tiff_file)[0]}_frame_{i:04d}.jpg'
                    frame_path = os.path.join(output_folder, frame_filename)
                    image = Image.fromarray(frame)
                    image.save(frame_path)
                    print(f'Saved frame {i} from {tiff_file} to {frame_path}')

tiff_folder = 'ucsd_dataset/Ped1/Test/Test001'  
output_folder = 'ucsd_dataset/Ped1/Test/frames'
extract_frames_from_tiff_folder(tiff_folder, output_folder)


Processing file: ucsd_dataset/Ped1/Test/Test001\001.tif
Saved frame 0 from 001.tif to ucsd_dataset/Ped1/Test/frames\001_frame_0000.jpg
Processing file: ucsd_dataset/Ped1/Test/Test001\002.tif
Saved frame 0 from 002.tif to ucsd_dataset/Ped1/Test/frames\002_frame_0000.jpg
Processing file: ucsd_dataset/Ped1/Test/Test001\003.tif
Saved frame 0 from 003.tif to ucsd_dataset/Ped1/Test/frames\003_frame_0000.jpg
Processing file: ucsd_dataset/Ped1/Test/Test001\004.tif
Saved frame 0 from 004.tif to ucsd_dataset/Ped1/Test/frames\004_frame_0000.jpg
Processing file: ucsd_dataset/Ped1/Test/Test001\005.tif
Saved frame 0 from 005.tif to ucsd_dataset/Ped1/Test/frames\005_frame_0000.jpg
Processing file: ucsd_dataset/Ped1/Test/Test001\006.tif
Saved frame 0 from 006.tif to ucsd_dataset/Ped1/Test/frames\006_frame_0000.jpg
Processing file: ucsd_dataset/Ped1/Test/Test001\007.tif
Saved frame 0 from 007.tif to ucsd_dataset/Ped1/Test/frames\007_frame_0000.jpg
Processing file: ucsd_dataset/Ped1/Test/Test001\008.tif

In [17]:
from glob import glob

def load_frames(frame_folder, target_size=(128, 128)):
    frame_files = sorted(glob(os.path.join(frame_folder, '*.jpg')))
    frames = []

    for frame_file in frame_files:
        img = Image.open(frame_file).resize(target_size)
        img_array = np.array(img)
        if img_array.ndim == 2:  # Convert grayscale to RGB
            img_array = np.stack([img_array] * 3, axis=-1)
        frames.append(img_array)

    frames = np.array(frames)
    print(f"Loaded {frames.shape[0]} frames with shape {frames.shape[1:]}")
    return frames

# Example usage
frame_folder = 'ucsd_dataset/Ped1/Test/frames'
frames = load_frames(frame_folder)
frames = frames.astype('float32') / 255.0  # Normalize pixel values
print("Shape of frames:", frames.shape)  # Should be (num_frames, height, width, channels)

Loaded 201 frames with shape (128, 128, 3)
Shape of frames: (201, 128, 128, 3)


In [18]:
# Normalize pixel values
frames = frames.astype('float32') / 255.0

input_shape = frames.shape[1:]  # Exclude the number of frames

input_img = Input(shape=input_shape)

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(input_shape[2], (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder Model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

# Split data
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test = train_test_split(frames, test_size=0.2, random_state=42)

# Train the Model
autoencoder.fit(X_train, X_train, epochs=50, batch_size=128, shuffle=True, validation_data=(X_test, X_test))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_6 (UpSampling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_7 (UpSampling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 128, 128, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,651 (295.51 KB)

 Trainable params: 75,651 (295.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 0.6931 - val_loss: 0.6894
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 498ms/step - loss: 0.6890 - val_loss: 0.6826
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 505ms/step - loss: 0.6820 - val_loss: 0.6708
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 405ms/step - loss: 0.6696 - val_loss: 0.6503
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 417ms/step - loss: 0.6482 - val_loss: 0.6160
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 501ms/step - loss: 0.6128 - val_loss: 0.5618
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 469ms/step - loss: 0.5570 - val_loss: 0.4816
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 412ms/step - loss: 0.4749 - val_loss: 0.3737
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 409ms/step - loss: 0.3654 - val_loss: 0.2485
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 418ms/step - loss: 0.2404 - val_loss: 0.1337
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 411ms/step - loss: 0.1279 - val_loss: 0.0587
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 421ms/step - loss: 0.0560 - val_loss:

In [19]:
# Predict and Calculate Reconstruction Error
decoded_imgs = autoencoder.predict(X_test)
reconstruction_error = np.mean(np.square(X_test - decoded_imgs), axis=(1, 2, 3))

# Define threshold for anomaly detection
threshold = np.percentile(reconstruction_error, 95)

# Predict anomalies
predicted_anomalies = (reconstruction_error > threshold).astype(int)

# Note: You need labels for y_test to evaluate accuracy, precision, and recall
# Here we assume you have labels and y_test for evaluation
# Example labels for the sake of demonstration:
y_test = np.zeros(len(X_test))  # Replace with actual labels

# Evaluation
accuracy = accuracy_score(y_test, predicted_anomalies)
precision = precision_score(y_test, predicted_anomalies)
recall = recall_score(y_test, predicted_anomalies)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 307ms/step
Accuracy: 0.95
Precision: 0.00
Recall: 0.00


C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
# Provide Recommendations for Response Protocols
def recommend_response(predicted_anomalies, frame_indices):
    responses = []
    for i, anomaly in enumerate(predicted_anomalies):
        if anomaly == 1:
            responses.append((frame_indices[i], 'Alert Security Personnel'))
        else:
            responses.append((frame_indices[i], 'No Action Needed'))
    return responses

# Example usage
frame_indices = range(len(predicted_anomalies))  # Replace with actual frame indices
responses = recommend_response(predicted_anomalies, frame_indices)

# Print recommendations
for frame_idx, response in responses:
    print(f'Frame {frame_idx}: {response}')


Frame 0: No Action Needed
Frame 1: No Action Needed
Frame 2: No Action Needed
Frame 3: No Action Needed
Frame 4: No Action Needed
Frame 5: No Action Needed
Frame 6: No Action Needed
Frame 7: No Action Needed
Frame 8: No Action Needed
Frame 9: No Action Needed
Frame 10: No Action Needed
Frame 11: No Action Needed
Frame 12: No Action Needed
Frame 13: No Action Needed
Frame 14: No Action Needed
Frame 15: No Action Needed
Frame 16: No Action Needed
Frame 17: No Action Needed
Frame 18: No Action Needed
Frame 19: No Action Needed
Frame 20: No Action Needed
Frame 21: No Action Needed
Frame 22: Alert Security Personnel
Frame 23: Alert Security Personnel
Frame 24: No Action Needed
Frame 25: No Action Needed
Frame 26: No Action Needed
Frame 27: No Action Needed
Frame 28: No Action Needed
Frame 29: No Action Needed
Frame 30: No Action Needed
Frame 31: No Action Needed
Frame 32: No Action Needed
Frame 33: No Action Needed
Frame 34: No Action Needed
Frame 35: No Action Needed
Frame 36: No Action Ne

### Conclusion

In this case study, we aimed to enhance security surveillance by developing a video analytics solution to automatically detect unusual activities or anomalies in real-time.
